# Live DemoIn this notebook we'll use the model we trained to detect whether the robot is ``free``, ``turn left``, `` turn right`` or ``blocked`` to enable a collision avoidance behavior on the robot.  

In [1]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 4)
model.load_state_dict(torch.load('i/best_model.pth'))
device = torch.device('cuda')
model = model.to(device)
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

from jetbot import Robot

robot = Robot()
def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
#blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
#speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

#display(widgets.VBox(image blocked_slider]), speed_slider]))


In [2]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    with torch.no_grad():
        y = model(x)
    
    classes = ['blocked','free', 'left','right']
    _, predicted = torch.max(y, 1)
    percentage = torch.softmax(y, dim=1)[0]*100
    
    print(f'{classes[predicted[0]]}, {percentage[predicted[0]]}.item()')
    
    if classes[predicted[0]]=='blocked':
        robot.backward(0.4)
        time.sleep(0.1)
    elif classes[predicted[0]]=='left':
        robot.left(0.4)
        time.sleep(0.1)
    elif classes[predicted[0]]=='right':
        robot.right(0.4)
        time.sleep(0.1)
    elif classes[predicted[0]]=='forward':
        robot.forward(0.4)
        time.sleep(0.1)
    else:
        robot.stop()
    
        
    time.sleep(0.001)
        
display(widgets.VBox([image]))
update({'new': camera.value})  # we call the function once to initialize
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

# Before you finish
You must unattach the callback and stop the camera by executing the code below before you finish all tasks.

In [3]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()
camera.stop()

### Conclusion

That's it for this live demo!  Hopefully you had some fun and your robot avoided collisions intelligently! 

If your robot wasn't avoiding collisions very well, try to spot where it fails.  The beauty is that we can collect more data for these failure scenarios
and the robot should get even better :)